# **Análise Exploratória da Base de Dados**

Nesse notebook será realizada uma análise exploratória do arquivo "ml_project1_data.csv", que contém os dados da campanha piloto com 2.240 instâncias, registrando as características dos clientes, e a respectiva resposta — se comprou ou não o produto.

Com essa análise, teremos um entendimento _data-driven_ sobre características dos clientes que o fazem mais rentáveis.

In [100]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [101]:
df = pd.read_csv(filepath_or_buffer='ml_project1_data.csv')

In [102]:
df.head()

ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0  2012-09-04       58       635  ...                  7             0   
1  2014-03-08       38        11  ...                  5             0   
2  2013-08-21       26       426  ...                  4             0   
3  2014-02-10       26        11  ...                  6             0   
4  2014-01-19       94       173  ...                  5             0   

   AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  Complain  \
0             0             0             0             0         0   
1             0             0             0             0         0   
2             0             0             0             0         0   
3             0             0             0             0         0   
4             0             0             0             0         0   

   Z_CostContact  Z_Revenue  Response  
0              3         11         1  
1              3         11         0  
2              3         11         0  
3              3         11         0  
4              3         11         0  

[5 rows x 29 columns]

## Tamanho da base

In [8]:
print('Formato da base (número de linhas x colunas):')
df.shape

Formato da base (número de linhas x colunas):


(2240, 29)

In [45]:
print('Conferindo casos de IDs duplicados:')

print('Duplicados: %.2f%%'%((len(df)-len(df.drop_duplicates(subset=['ID'])))/len(df)) )

Conferindo casos de IDs duplicados:
Duplicados: 0.00%


## Features

In [16]:
print('Número de features:', len(features))

Número de features: 28


In [103]:
keys = ['ID']
target = ['Response']

features = [ col for col in list(df) if col not in keys+target ]

In [104]:
print('Conferindo cardinalidade e filtrando features "sem informação"')
features_sem_cardinalidade = df.nunique()[df.nunique()==1].index

features = [ col for col in features if col not in features_sem_cardinalidade ]

print()
print('Features removidas:', list(features_sem_cardinalidade))
print('Novo número de features:', len(features))

Conferindo cardinalidade e filtrando features "sem informação"

Features removidas: ['Z_CostContact', 'Z_Revenue']
Novo número de features: 25


In [105]:
df.drop(columns=features_sem_cardinalidade, inplace=True)

## Análise do Alvo

In [106]:
print('Taxa de conversão:', )
100*df['Response'].value_counts(normalize=True)

Taxa de conversão:


0    85.089286
1    14.910714
Name: Response, dtype: float64

## Tipos das Variáveis

In [107]:
num_features = list(df[features]._get_numeric_data())

cat_features = [ col for col in features if col not in num_features ]

In [108]:
print('Qtd de variáveis numéricas:', len(num_features))
print(num_features)

print()

print('Qtd de variáveis categóricas:', len(cat_features))
print(cat_features)

Qtd de variáveis numéricas: 22
['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain']

Qtd de variáveis categóricas: 3
['Education', 'Marital_Status', 'Dt_Customer']


## Construção de novas variáveis

In [109]:
data_atual = '2014-10-01'
ano_atual = int(data_atual[:4])

data_atual = datetime.strptime(data_atual, '%Y-%m-%d')

In [110]:
# Idade - Irá substituir o ano de nascimento, para um melhor ideia mais clara de conceito
df['Age'] = (ano_atual - df['Year_Birth'])

# Tempo do cliente na empresa - Irá substituir a data de entrada, pois será melhor trabalhada nos algoritmos (clusterização e predição)
df['Time_Customer'] = (data_atual - pd.to_datetime(df['Dt_Customer'])).dt.days/365

# Somatório de filhos (Kids + Teen)
df['SonsTotal'] = df[['Kidhome', 'Teenhome']].sum(axis=1)

# Total Gasto (Somatório de produtos)
df['MntTotal'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts']].sum(axis=1)

# Compras pelos canais:
df['NumPurchases'] = df[['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']].sum(axis=1)

# Média de Gasto Por Compra
df['MntPerPurchase'] = df['MntTotal']/df['NumPurchases']

# Variáveis relacionadas às campanhas anteriores
df['AcceptedCmp_Anterior'] = (df[features_campanha].sum(axis=1)>0).astype(int)
df['AcceptedCmp_Total'] = df[features_campanha].sum(axis=1)

novas_features = [ col for col in list(df) if col not in keys+features+target ]
#novas_vars = ['Age', 'Time_Customer', 'MntTotal', 'NumPurchases', 'SonsTotal', 'AcceptedCmp_Anterior', 'AcceptedCmp_Total']

In [111]:
features = features+novas_features

## Divisão das colunas e seus conceitos

In [112]:
# Características obtidas no cadastro do cliente no sistema
features_cadastro = ['Age', 'Time_Customer' 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'SonsTotal']

# Características comportamentais, que são atualizadas no decorrer do histórico de relacionamento do cliente
features_comportamento = ['Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'MntTotal', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumPurchases', 'NumWebVisitsMonth', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Complain']

# Características relacionadas ao canal de compra
features_canal = ['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']

# Características relacionadas às campanhas 
features_campanha = [ col for col in list(df) if col.startswith('AcceptedCmp') ]

## Análise Univariada

1. Missing
2. Cardinalidade

In [113]:
df_infos = pd.DataFrame(columns=['MISSING', 'CARDINALIDADE'], index=list(df))

df_infos['MISSING'] = 100*df.isnull().sum()/len(df)
df_infos['CARDINALIDADE'] = df.nunique()

df_infos

MISSING  CARDINALIDADE
ID                    0.000000           2240
Year_Birth            0.000000             59
Education             0.000000              5
Marital_Status        0.000000              8
Income                1.071429           1974
Kidhome               0.000000              3
Teenhome              0.000000              3
Dt_Customer           0.000000            663
Recency               0.000000            100
MntWines              0.000000            776
MntFruits             0.000000            158
MntMeatProducts       0.000000            558
MntFishProducts       0.000000            182
MntSweetProducts      0.000000            177
MntGoldProds          0.000000            213
NumDealsPurchases     0.000000             15
NumWebPurchases       0.000000             15
NumCatalogPurchases   0.000000             14
NumStorePurchases     0.000000             14
NumWebVisitsMonth     0.000000             16
AcceptedCmp3          0.000000              2
AcceptedCmp4          0.000000              2
AcceptedCmp5          0.000000              2
AcceptedCmp1          0.000000              2
AcceptedCmp2          0.000000              2
Complain              0.000000              2
Response              0.000000              2
Age                   0.000000             59
Time_Customer         0.000000            663
SonsTotal             0.000000              4
MntTotal              0.000000            902
NumPurchases          0.000000             33
AcceptedCmp_Anterior  0.000000              2
AcceptedCmp_Total     0.000000              5

## Salvando o dataframe com modificações

In [115]:
df.to_csv(path_or_buf='ml_project1_data_NOVO.csv')